## Important Imports

In [44]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import math

'''
import tensorflow as tp
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from keras import backend as K
import math
import matplotlib.pyplot as plt
import os
from tensorflow.keras import regularizers 
'''
import statsmodels.api as sm  

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.utils.fixes import _parse_version
from sklearn.preprocessing import QuantileTransformer, quantile_transform
from sklearn.impute import SimpleImputer

from sklearn.metrics import accuracy_score as acc
#from mlxtend.feature_selection import SequentialFeatureSelector as sfs

## Data Preprocessing

In [77]:
AutoMPG = pd.read_csv("auto-mpg.csv", header = 1)
Concrete = pd.read_csv("concrete.csv", header = 1)
ForestFires = pd.read_csv("forestfires.csv", header = 1)
SkillCraft = pd.read_csv("SkillCraft1_Dataset.csv", header = 1)
BiasCorrect = pd.read_csv("Bias_correction_ucl.csv", header = 1)

# X_AutoMPG = AutoMPG.drop(columns = ['model-year'])
# Y_AutoMPG = AutoMPG[['model-year']]

# X_Concrete = Concrete.drop(columns = ['Compressive Strength (28-day)(Mpa)'])
# Y_Concrete = Concrete[['Compressive Strength (28-day)(Mpa)']]

# X_ForestFires = ForestFires.drop(columns = ['month','day','area'])
# Y_ForestFires = ForestFires[['area']]

# X_SkillCraft = SkillCraft.drop(columns = ['ComplexAbilitiesUsed'])
# Y_SkillCraft = SkillCraft[['ComplexAbilitiesUsed']]

# X_BiasCorrect = BiasCorrect.drop(columns = ['Next_Tmin'])
# Y_BiasCorrect = BiasCorrect[['Next_Tmin']]



# X = X_AutoMPG.fillna(X_AutoMPG.mean())
# y = Y_AutoMPG.fillna(Y_AutoMPG.mean())
    
# X_train, X_test, y_train, y_test = train_test_split(X_AutoMPG, Y_AutoMPG, random_state=1)
# y_train.values
# X_train.to_numpy()
# y_test.to_numpy()
# X_test.to_numpy()
# print(y_train)

# dataset = AutoMPG.to_numpy()
# Y = dataset[:, -1]
# X = dataset[:, :-1]
    
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)   
    
# print(X_train[:,[0,1,2,3,4,5]])
# print([0,1]+[3])

[[2.400e+01 6.000e+00 2.000e+02 8.100e+01 3.012e+03 1.760e+01]
 [9.000e+00 8.000e+00 3.040e+02 1.930e+02 4.732e+03 1.850e+01]
 [1.200e+01 8.000e+00 4.290e+02 1.980e+02 4.952e+03 1.150e+01]
 ...
 [1.300e+01 8.000e+00 3.070e+02 1.300e+02 4.098e+03 1.400e+01]
 [2.550e+01 4.000e+00 1.400e+02 8.900e+01 2.755e+03 1.580e+01]
 [1.400e+01 8.000e+00 3.500e+02 1.650e+02 4.209e+03 1.200e+01]]
[0, 1, 3]


## Transformed Regression with Ridge Regression

In [48]:
def TranReg_Ridge_Test(X, y): 
    f, (ax0, ax1) = plt.subplots(2, 2, sharey='row', figsize=(6.5, 8))
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

    regr = RidgeCV()
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)

    ax0[0].scatter(y_pred, y_test, s=8)
    ax0[0].plot([0, 7e5], [0, 7e5], '--k')
    ax0[0].set_ylabel('True target')
    ax0[0].set_xlabel('Predicted target')
    ax0[0].text(s='Ridge regression \n without target transformation', x=-5e4,
                y=8e5, fontsize=12, multialignment='center')
    ax0[0].text(3e4, 64e4, r'$R^2$=%.2f, MAE=%.2f' % (
        r2_score(y_test, y_pred), median_absolute_error(y_test, y_pred)))
    ax0[0].set_xlim([0, 7e5])
    ax0[0].set_ylim([0, 7e5])
    ax0[0].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

    ax1[0].scatter(y_pred, (y_pred - y_test), s=8)
    ax1[0].set_ylabel('Residual')
    ax1[0].set_xlabel('Predicted target')
    ax1[0].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

    regr_trans = TransformedTargetRegressor(regressor=RidgeCV(), 
                                            func=np.log, inverse_func=np.exp)
    regr_trans.fit(X_train, y_train)
    y_pred = regr_trans.predict(X_test)

    ax0[1].scatter(y_pred, y_test, s=8)
    ax0[1].plot([0, 7e5], [0, 7e5], '--k')
    ax0[1].set_ylabel('True target')
    ax0[1].set_xlabel('Predicted target')
    ax0[1].text(s='Ridge regression \n with target transformation', x=-5e4,
                y=8e5, fontsize=12, multialignment='center')
    ax0[1].text(3e4, 64e4, r'$R^2$=%.2f, MAE=%.2f' % (
        r2_score(y_test, y_pred), median_absolute_error(y_test, y_pred)))
    ax0[1].set_xlim([0, 7e5])
    ax0[1].set_ylim([0, 7e5])
    ax0[1].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

    ax1[1].scatter(y_pred, (y_pred - y_test), s=8)
    ax1[1].set_ylabel('Residual')
    ax1[1].set_xlabel('Predicted target')
    ax1[1].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

    f.suptitle("AutoMPG model year", y=0.035)

    plt.show()

## Transformed Regression with Lasso Regression

In [60]:
def TranReg_Lasso_Test(X, y): 
    f, (ax0, ax1) = plt.subplots(2, 2, sharey='row', figsize=(6.5, 8))
    X = X.fillna(X.mean())
    y = y.fillna(y.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    y_train.to_numpy()
    X_train.to_numpy()
    y_test.to_numpy()
    X_test.to_numpy()

    regr = Lasso()
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)

    ax0[0].scatter(y_pred, y_test, s=8)
    ax0[0].plot([0, 7e5], [0, 7e5], '--k')
    ax0[0].set_ylabel('True target')
    ax0[0].set_xlabel('Predicted target')
    ax0[0].text(s='Lasso regression \n without target transformation', x=-5e4,
                y=8e5, fontsize=12, multialignment='center')
    ax0[0].text(3e4, 64e4, r'$R^2$=%.2f, MAE=%.2f' % (
        r2_score(y_test, y_pred), median_absolute_error(y_test, y_pred)))
    ax0[0].set_xlim([0, 7e5])
    ax0[0].set_ylim([0, 7e5])
    ax0[0].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

    ax1[0].scatter(y_pred.T, (y_pred.T - y_test.T), s=8)
    ax1[0].set_ylabel('Residual')
    ax1[0].set_xlabel('Predicted target')
    ax1[0].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

    regr_trans = TransformedTargetRegressor(regressor=RidgeCV(),
                                            transformer=QuantileTransformer(n_quantiles=10,
                                                                            output_distribution='normal'))
    regr_trans.fit(X_train, y_train)
    y_pred = regr_trans.predict(X_test)

    ax0[1].scatter(y_pred, y_test, s=8)
    ax0[1].plot([0, 7e5], [0, 7e5], '--k')
    ax0[1].set_ylabel('True target')
    ax0[1].set_xlabel('Predicted target')
    ax0[1].text(s='Lasso regression \n with target transformation', x=-5e4,
                y=8e5, fontsize=12, multialignment='center')
    ax0[1].text(3e4, 64e4, r'$R^2$=%.2f, MAE=%.2f' % (
        r2_score(y_test, y_pred), median_absolute_error(y_test, y_pred)))
    ax0[1].set_xlim([0, 7e5])
    ax0[1].set_ylim([0, 7e5])
    ax0[1].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

    ax1[1].scatter(y_pred.T, (y_pred.T - y_test.T), s=8)
    ax1[1].set_ylabel('Residual')
    ax1[1].set_xlabel('Predicted target')
    ax1[1].ticklabel_format(axis="both", style="sci", scilimits=(0, 0))

    f.suptitle("AutoMPG model year", y=0.035)

    plt.show()

## Transformed Regression with forward selection

In [69]:
'''
X_train, X_test, y_train, y_test = train_test_split(
    df.values[:,:-1],
    df.values[:,-1:],
    test_size=0.25,
    random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()
'''    

regr_trans = TransformedTargetRegressor(regressor=LinearRegression(),
                                        func=np.log, inverse_func=np.exp)



In [70]:
def forward_selection(data): 
    dataset = data.to_numpy()
    Y = dataset[:, -1]
    X = dataset[:, :-1]
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)   
    
    m,n=X.shape
    
    X_cols = [] # buffer list that tells which column index is in X right now...
    X_cols_cv=[]
    X_set=np.empty((m,0))#for test and train with cv..this would need to be different..
    #need to save the below values for all models calculated in for loop and only save the best sse's
    
    m_train,n_train=X_train.shape
    m_test,n_test=X_test.shape
    X_train_set=np.empty((m_train,0))
    X_test_set=np.empty((m_test,0))
    
    r_square=[]
    r_adj=[]
    r_square_cv=[]
    
    while (len(X_cols)<n):
        sse=[]
        r_square_best=[]
        r_adj_best=[]
        r_square_cv_best=[]
        for i in range(0, n) :
            if i not in X_cols :
                #for rsq and radjsq
                mno,nno=X_train_set.shape
                X_train_set_copy=X_train[:, X_cols+[i]]
                regr_trans.fit(X_train_set_copy, Y_train)
                y_pred = regr_trans.predict(X_test)
            
                X_test_set_copy=X_test[:,X_cols+[i]]
                r2 = r2_score(Y_test, y_pred)
                r2_adj = 1-(1-r2)*(m-1)/(m-mno-1)
                r_square_best.append(r2)
                r_adj_best.append(r2_adj)                
                r_square_cv_best.append(cross_val_score(regr_trans, X_train_set_copy, Y_train, cv=1))
                
                sse.append(mean_squared_error(Y_Test, y_pred))
                
            else: 
                sse.append(math.inf)#to
                r_square_best.append(float('-inf'))
                r_adj_best.append(float('-inf'))                
                r_square_cv_best.append(float('-inf'))
                
        #for rsq and radjsq
        best_index=sse.index(min(sse))
        X_cols.append(best_index)
        r_square_cv.append(r_square_cv_best[best_index])
        r_adj_cv.append(r_adj_cv_best[best_index])
        r_square.append(r_square_best[best_index])
        r_adj.append(r_adj_best[best_index])

    return r_square,r_adj,r_square_cv


In [79]:
forward_selection(AutoMPG)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 6)

In [78]:
def backward_elimination(data):
    dataset = data.to_numpy()
    Y = dataset[:, -1]
    X = dataset[:, :-1]
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=1)   
    
    m,n=X.shape
    X_cols = [] # buffer list that tells which column index is in X right now...
    for j in range(n):
        X_cols += [j]
    X_set = np.empty((m,0))#for test and train with cv..this would need to be different..
    #need to save the below values for all models calculated in for loop and only save the best sse's
    
    m_train,n_train=X_train.shape
    m_test,n_test=X_test.shape
    X_train_set=np.empty((m_train,0))
    X_test_set=np.empty((m_test,0))
    
    r_square=[]
    r_adj=[]
    r_square_cv=[]
    
    while (len(X_cols)<n):
        sse=[]
        r_square_best=[]
        r_adj_best=[]
        r_square_cv_best=[]
        for i in range(n-1,0,-1) :
            if i not in X_cols :
                #for rsq and radjsq
                mno,nno=X_train_set.shape
                X_train_set_copy=X_train[:, X_cols-[i]]
                regr_trans.fit(X_train_set_copy,Y_train)
                y_pred = regr_trans.predict(X_test)
            
                X_test_set_copy=X_test[:, X_cols-[i]]
                r2 = r2_score(Y_test, y_pred)
                r2_adj = 1-(1-r2)*(m-1)/(m-mno-1)
                r_square_best.append(r2)
                r_adj_best.append(r2_adj)                
                r_square_cv_best.append(cross_val_score(regr_trans, X_train_set_copy, Y_train, cv=1))
                
                sse.append(mean_squared_error(Y_Test, y_pred))
                
            else: 
                sse.append(math.inf)#to
                r_square_best.append(float('-inf'))
                r_adj_best.append(float('-inf'))                
                r_square_cv_best.append(float('-inf'))
                
        #for rsq and radjsq
        best_index=sse.index(min(sse))
        X_cols.append(best_index)
        r_square_cv.append(r_square_cv_best[best_index])
        r_adj_cv.append(r_adj_cv_best[best_index])
        r_square.append(r_square_best[best_index])
        r_adj.append(r_adj_best[best_index])

    return r_square,r_adj,r_square_cv
